In [1]:
import pandas as pd
df = pd.read_csv("full_data_fusionne.csv")
df["tags"].value_counts().to_dict()

{'NO_TAGS': 10082,
 'أخبار إسرائيل': 4197,
 'الأخبار المغاربية': 3229,
 'الشرق الأوسط': 2209,
 'أخبار أميركا': 1828,
 'أخبار فلسطين': 1775,
 'أوروبا': 1038,
 'أخبار لبنان': 924,
 'شرق أوسط': 909,
 'إسرائيل': 887,
 'أمريكا': 759,
 'ترامب': 656,
 'رياضة': 624,
 'فرنسا': 577,
 'أفريقيا': 526,
 'اقتصاد': 496,
 'أخبار إيران': 411,
 'سوريا': 402,
 'أخبار العالم': 351,
 'أخبار سوريا': 339,
 'غزة': 331,
 'آسيا': 323,
 'أزمة أوكرانيا': 299,
 'نفط': 285,
 'مجتمع': 275,
 'ذهب': 264,
 'أخبار روسيا': 216,
 'أسواق عالمية': 210,
 'أسواق': 191,
 'الأسد': 187,
 'أخبار الشركات': 177,
 'أخبار اليمن': 177,
 'أخبار مصر': 165,
 'منوعات': 141,
 'عملات': 141,
 'إسرائيل غزة': 132,
 'إسرائيل الإسرائيلي الجيش الإسرائيلي': 132,
 'الأميركية الأميركي': 125,
 'ثقافة': 124,
 'Businessمع لبنى': 118,
 'أميركا': 115,
 'إسرائيل الإسرائيلي': 111,
 'تكنولوجيا': 103,
 'إسرائيل الإسرائيلي الإسرائيلية': 102,
 'دونالد ترامب': 100,
 'اقتصاد عالمي': 99,
 'الأميركي': 98,
 'طاقة': 91,
 'انتخابات أميركا': 90,
 'أخبار الصين': 84,
 '

In [11]:
# Remove rows with missing or empty tags
df = df[~df['tags'].isnull()]
df = df[df['tags'].str.strip() != '']


In [13]:
def clean_tags(tag_string):
    tags = tag_string.split(',')
    tags = [t.strip().lower() for t in tags]  # remove spaces and lowercase
    tags = [t for t in tags if t not in ['', 'no_tags', 'none']]  # remove junk tags
    return tags

df['cleaned_tags'] = df['tags'].apply(clean_tags)


In [15]:
df = df[df['cleaned_tags'].map(len) > 0]


In [17]:
from collections import Counter

all_tags_flat = [tag for sublist in df['cleaned_tags'] for tag in sublist]
tag_counter = Counter(all_tags_flat)


In [19]:
tag_counter

Counter({'أخبار إسرائيل': 4197,
         'الأخبار المغاربية': 3229,
         'الشرق الأوسط': 2209,
         'أخبار أميركا': 1828,
         'أخبار فلسطين': 1775,
         'أوروبا': 1038,
         'أخبار لبنان': 924,
         'شرق أوسط': 909,
         'إسرائيل': 887,
         'أمريكا': 759,
         'ترامب': 656,
         'رياضة': 624,
         'فرنسا': 577,
         'أفريقيا': 526,
         'اقتصاد': 496,
         'أخبار إيران': 411,
         'سوريا': 402,
         'أخبار العالم': 351,
         'أخبار سوريا': 339,
         'غزة': 331,
         'آسيا': 323,
         'أزمة أوكرانيا': 299,
         'نفط': 285,
         'مجتمع': 275,
         'ذهب': 264,
         'أخبار روسيا': 216,
         'أسواق عالمية': 210,
         'أسواق': 191,
         'الأسد': 187,
         'أخبار الشركات': 177,
         'أخبار اليمن': 177,
         'أخبار مصر': 165,
         'عملات': 141,
         'منوعات': 141,
         'إسرائيل غزة': 132,
         'إسرائيل الإسرائيلي الجيش الإسرائيلي': 132,
         'الأميركية ا

In [25]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict
import numpy as np

# Your dataframe: df['tags'] should be a list of strings per row
# First: flatten all tags into a single list
all_tags = df['cleaned_tags'].explode().dropna().unique().tolist()

# Load a multilingual model that works well with Arabic
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Step 1: Embed all tags
embeddings = model.encode(all_tags)

# Step 2: Cluster embeddings (change n_clusters or use distance_threshold for dynamic)
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.0)
clustering_model.fit(embeddings)

# Step 3: Map clustered tags to a representative tag (e.g., first tag in the group)
cluster_to_tags = defaultdict(list)
for idx, label in enumerate(clustering_model.labels_):
    cluster_to_tags[label].append(all_tags[idx])

# Choose a representative tag for each cluster
cluster_rep_map = {}
for label, tag_list in cluster_to_tags.items():
    rep = tag_list[0]  # You could improve this by picking the shortest or most common
    for tag in tag_list:
        cluster_rep_map[tag] = rep

# Step 4: Apply the mapping to the dataframe
def normalize_with_cluster(tags):
    return list({cluster_rep_map.get(tag, tag) for tag in tags if tag})

df['normalized_tags'] = df['cleaned_tags'].apply(normalize_with_cluster)


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (C:\Users\HP\anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py)

In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(["اختبار", "تجربة"])
print(embeddings)


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (C:\Users\HP\anaconda3\Lib\site-packages\tensorflow\python\framework\dtypes.py)